# NLP with Bert for Sentiment Analysis

### Importing the libraries

In [1]:
!pip3 install ktrain

     |████████████████████████████████| 25.3MB 129kB/s 
     |████████████████████████████████| 6.8MB 49.8MB/s 
     |████████████████████████████████| 983kB 52.4MB/s 
     |████████████████████████████████| 266kB 49.0MB/s 
     |████████████████████████████████| 1.9MB 47.0MB/s 
     |████████████████████████████████| 1.2MB 46.5MB/s 
     |████████████████████████████████| 471kB 46.3MB/s 
     |████████████████████████████████| 3.3MB 43.4MB/s 
     |████████████████████████████████| 870kB 46.7MB/s 
  Created wheel for ktrain: filename=ktrain-0.26.2-cp37-none-any.whl size=25277794 sha256=9cd7a0072d0197b4add46f6e576a0f5e641a96bc04ddb08d37b364077e917383
  Stored in directory: /root/.cache/pip/wheels/67/2e/f1/c72afa08df8b2d984b910dea228902ce81dae4511afe9fafd2
  Created wheel for langdetect: filename=langdetect-1.0.8-cp37-none-any.whl size=993193 sha256=9c2ec6e2f2445eba4f457ef62d606b13eb53c4b1a673e852bee825e1f26eae34
  Stored in directory: /root/.cache/pip/wheels/8d/b3/aa/6d99de9f3841d7d3d4

In [11]:
import os.path
import numpy as np
import tensorflow as tf
import ktrain # Ktrain is a wrapper for tensorflow.keras that allows to us to wrap some models in only a few lines of code
from ktrain import text

## Part 1: Data Preprocessing

### Loading the IMDB dataset

In [7]:
# get_file used to download a file/dataset from a url
dataset = tf.keras.utils.get_file(fname="aclImdb_v1.tar.gz",
                                  origin="http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz",
                                  extract=True)
IMDB_DATADIR = os.path.join(os.path.dirname(dataset), 'aclImdb') # gets the dir folder path and 

In [6]:
print(os.path.dirname(dataset)) # path to all the datasets
print(IMDB_DATADIR) # path to the imdb dataset

/root/.keras/datasets
/root/.keras/datasets/aclImdb


### Creating the training and test sets

In [10]:
# texts_from_folder returns (trn, val, preproc), where val is our test in this case
# since we know the function returns three items in a tuples, we destructure thosee items immediately
(x_train, y_train), (x_test, y_test), preproc = text.texts_from_folder(datadir=IMDB_DATADIR,
                                                                       classes=['pos','neg'],
                                                                       maxlen=500, #max number of word we going to use
                                                                       train_test_names=['train','test'],
                                                                       preprocess_mode='bert')

detected encoding: utf-8
downloading pretrained BERT model (uncased_L-12_H-768_A-12.zip)...
[██████████████████████████████████████████████████]
extracting pretrained BERT model...
done.

cleanup downloaded zip...
done.

preprocessing train...
language: en


Is Multi-Label? False
preprocessing test...
language: en


## Part 2: Building the BERT model

In [12]:
model = text.text_classifier(name='bert',
                             train_data=(x_train, y_train),
                             preproc=preproc)

Is Multi-Label? False
maxlen is 500
done.


## Part 3: Training the BERT model

In [15]:
# this gets the learner instance of the Bert model.

learner = ktrain.get_learner(model=model,
                             train_data=(x_train, y_train),
                             val_data=(x_test, y_test),
                             batch_size=6)


 <!-- Choosing the batch size -->

The sequence corresponds to the  maxlen used during creation of the test and train set

sequence len=64, max_batch_size=64
        sequence len=128, max_batch_size=32
        sequence len=256, max_batch_size=16
        sequence len=320, max_batch_size=14
        sequence len=384, max_batch_size=12
        sequence len=512, max_batch_size=6

In [16]:
# training

learner.fit_onecycle(lr=2e-5,
                     epochs=1)



begin training using onecycle policy with max lr of 2e-05...
4167/4167 [==============================] - 5406s 1s/step - loss: 0.3390 - accuracy: 0.8470 - val_loss: 0.1643 - val_accuracy: 0.9375
